# Shows examples of how to post-process data, including detrending and making maps for 1D, 2D, and 2D detail files

by Sanne Veldhuisjen, Tesse van den Aker, and Elizbeth Case

In [1]:
import os
import glob
import pandas as pd
import xarray as xr
import numpy as np
import math
import datetime as dt
from datetime import timedelta, datetime


In [ ]:
# set working directory
#os.chdir('/perm/nld3562/code/IMAU-FDM/post-process/make_1d_files/')

# settings 
var = "FirnAir"           # FirnAir (can be any output from 1D files)
if var=="FirnAir" or var=="dz":
    dt = 1  		# 0=no detrending, 1=detrending --> Should be done for "FirnAir" and "zs" (include in file name if not)
else:
    dt=0
    
atzero = 1  		# 0=start at zero,1=do not start at zero --> (Actually I realized I do not use this anymore, because first output is after 10 days...) But could be used for "zs".)
nyears = 84 		# amount of years 
ts_su = 10957 		# timestep spin up: 1533 for Antarctica 1979-2020, 10950 for 30 year greenland spinup using 365 dpy, or 10957 using 365.25 dpy

# directories and files
project_name = "run_FGRN055-era055_1939-2023"
code_dir = "/perm/nld4814/code/IMAU-FDM/"
project_dir = "/ec/res4/scratch/nld4814/"

pointlist_dir = f"{project_dir}pointlist_1.txt"        # pointlist used for run
mask_dir = f"{code_dir}reference/FGRN055/"          # directory for mask 
data_dir = f"{project_dir}{project_name}output/"           # directory for 1D output files 
output_dir = f"{project_dir}{project_name}post-process/"     # directory save gridded file  
os.makedirs(f"{dir4}", exist_ok=True)

pre_fname1 = "FGRN055_era055"	  		# file name 1D output files 
pointlist_file = pointlist_dir + "IN_ll_FGRN055.txt"    # pointlist 
pointlist_df = pd.read_csv(pointlist_file,header=None)			# open pointlist 
nof = len(pointlist_df) 					# number of files 

if dt==1:
    f_detrended = "detrended"
else:
    f_detrended = ""

fout= f"{output_dir}FDMv12G_" + variab + "_1939-2023_ERA5_FGRN055_{f_detrended}.nc" 


In [ ]:
# post-processing fdm data by point, adapted from Sanne's scripts

from datetime import datetime,timedelta

domain = "FGRN055"
var = "dens"

start_date = datetime(1939,9,1)
end_date = datetime(2023,12,31)
nyears = end_date.year - start_date.year	# amount of years 1940-2023, not including last part of 1939 (1939.66575)
ndays_timestep = 10      # in days, of shortest write time (usually 1 or 10 days, set in start_model_ccab in sec)

fdm_index = 36618
run_name = "cores"
dir_model_output = "/ec/res4/scratch/nld4814/"+run_name+"/output/"
dir_mask = "/perm/nld4814/code/IMAU-FDM/reference/" + domain + "/"

#Import 1D, 2D, and 2Ddetail data
fdm_1D = xr.open_dataset(dir_model_output+"FGRN055_era055_1D_"+str(fdm_index)+".nc")
fdm_2D = xr.open_dataset(dir_model_output+"FGRN055_era055_2D_"+str(fdm_index)+".nc")
fdm_2D_detail = xr.open_dataset(dir_model_output+"FGRN055_era055_2Ddetail_"+str(fdm_index)+".nc")

# create timestep dimension

def create_fdm_datetime(df, start_date, end_date, ndays_timestep):

    ##################
    
    # df: fdm data output (1d, 2d, 2ddetail)
    # start_date: start date of dataset
    # end_date: end date of dataset
    # ndays_timestep: output frequency in days (usually 1, 10, or 30 days, specified in start_model_ccab)


    # Initialize an empty list
    date_list = []
    start_date += timedelta(days=ndays_timestep)
     
    # Loop through the range of dates and append to the list
    while start_date <= end_date:
        date_list.append(start_date)
        start_date += timedelta(days=ndays_timestep)
